In [56]:
import pandas as pd

import numpy as np
np.random.seed(0)

import matplotlib.pyplot as plt
%matplotlib inline

import nltk

from nltk.tokenize import RegexpTokenizer
from nltk.probability import FreqDist

import string

import re

from nltk.stem import WordNetLemmatizer

import tensorflow as tf

from gensim.models import Word2Vec



ImportError: Traceback (most recent call last):
  File "C:\Users\kas2n\anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 64, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: The specified module could not be found.


Failed to load the native TensorFlow runtime.

See https://www.tensorflow.org/install/errors

for some common reasons and solutions.  Include the entire stack trace
above this error message when asking for help.

In [4]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kas2n\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kas2n\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kas2n\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
df = pd.read_csv('data/sentiment_tweets3.csv')

In [47]:
df.head(20)

,Index,message to examine,label (depression result),tokenized,lemmatized
0,106,just had a real good moment. i missssssssss hi...,0,"[just, had, a, real, good, moment, ., i, misss...",just had a real good moment. i missssssssss hi...
1,217,is reading manga http://plurk.com/p/mzp1e,0,"[is, reading, manga]",is reading manga
2,220,@comeagainjen http://twitpic.com/2y2lx - http:...,0,[-],@comeagainjen -
3,288,@lapcat Need to send 'em to my accountant tomo...,0,"[need, to, send, ', em, to, my, accountant, to...",@lapcat need to send 'em to my accountant tomo...
4,540,ADD ME ON MYSPACE!!! myspace.com/LookThunder,0,"[add, me, on, myspace, !, !, !]",add me on myspace!!!
5,624,so sleepy. good times tonight though,0,"[so, sleepy, ., good, times, tonight, though]",so sleepy. good times tonight though
6,701,"@SilkCharm re: #nbn as someone already said, d...",0,"[re, :, #nbn, as, someone, already, said, ,, d...","@silkcharm re: #nbn as someone already said, d..."
7,808,23 or 24ï¿½C possible today. Nice,0,"[23, or, 24ï, ¿, ½, c, possible, today, ., nice]",23 or 24ï¿½c possible today. nice
8,1193,nite twitterville workout in the am -ciao,0,"[nite, twitterville, workout, in, the, am, -, ...",nite twitterville workout in the am -ciao
9,1324,"@daNanner Night, darlin'! Sweet dreams to you",0,"[night, ,, darlin, ', !, sweet, dreams, to, you]","@dananner night, darlin'! sweet dreams to you"


In [10]:
df['label (depression result)'].value_counts(normalize = True)

0    0.775645
1    0.224355
Name: label (depression result), dtype: float64

In [11]:
df.shape

(10314, 3)

In [24]:
df.drop_duplicates(inplace = True)
df.shape

(10314, 3)

In [16]:
df.isna().any()

Index                        False
message to examine           False
label (depression result)    False
dtype: bool

In [6]:
from nltk.tokenize import TweetTokenizer
tweet_tokenizer = TweetTokenizer(strip_handles = True, reduce_len=True)

tweet_tokens = []
for sent in df['message to examine']:
    tweet_tokens.append(tweet_tokenizer.tokenize(sent))
    
tweet_tokens[:20]


[['just',
  'had',
  'a',
  'real',
  'good',
  'moment',
  '.',
  'i',
  'misss',
  'him',
  'so',
  'much',
  ','],
 ['is', 'reading', 'manga', 'http://plurk.com/p/mzp1e'],
 ['http://twitpic.com/2y2lx',
  '-',
  'http://www.youtube.com/watch?v=zoGfqvh2ME8'],
 ['Need',
  'to',
  'send',
  "'",
  'em',
  'to',
  'my',
  'accountant',
  'tomorrow',
  '.',
  'Oddly',
  ',',
  'I',
  "wasn't",
  'even',
  'referring',
  'to',
  'my',
  'taxes',
  '.',
  'Those',
  'are',
  'supporting',
  'evidence',
  ',',
  'though',
  '.'],
 ['ADD', 'ME', 'ON', 'MYSPACE', '!', '!', '!', 'myspace.com/LookThunder'],
 ['so', 'sleepy', '.', 'good', 'times', 'tonight', 'though'],
 ['re',
  ':',
  '#nbn',
  'as',
  'someone',
  'already',
  'said',
  ',',
  'does',
  'fiber',
  'to',
  'the',
  'home',
  'mean',
  'we',
  'will',
  'all',
  'at',
  'least',
  'be',
  'regular',
  'now'],
 ['23', 'or', '24ï', '¿', '½', 'C', 'possible', 'today', '.', 'Nice'],
 ['nite', 'twitterville', 'workout', 'in', 'the', '

In [23]:
    test = 'is reading manga http://plurk.com/p/mzp1e'
    tweett = ' '.join(re.sub("http://\S+", " ", test).split())
    tweett


'is reading manga'

In [39]:
def clean_text(tweet):
    #remove numbers
    #text_nonum = re.sub(r'\d+', '', tweet)
    #remove hashtags
    text_nonum = re.sub('#', "", tweet)
    #remove punctuations and convert characters to lower case
    text_nopunct = "".join([char.lower() for char in text_nonum if char not in string.punctuation])
    #remove multiple whitespace and leading/trailing whitespace
    text_no_doublespace = re.sub('\s+', ' ', text_nopunct).strip()
    return text_no_doublespace

In [40]:
def nopunct(tweet):
    new_words = [word for word in tweet if word.isalnum()]
    return new_words

In [41]:
#def tokenize_tweets(tweets):

    
#remove URLS:
df['tokenized'] = [' '.join(re.sub("http://\S+", " ", tweet).split()) for tweet in df['message to examine']]
df['tokenized'] = [' '.join(re.sub("myspace.com\S+", " ", tweet).split()) for tweet in df['tokenized']]
#df['tokenized'] = [tweet.lower() for tweet in df['tokenized']]


#for tweet in df['tokenized']:
 #   df['tokenized'] = df['tokenized'].''.join([c for c in tweet if c not in punct])

#remove numbers
df['tokenized'] = [' '.join(re.sub(r'\d+', '', tweet).split()) for tweet in df['tokenized']]

#remove hashtag
df['tokenized'] = [' '.join(re.sub('#', "", tweet).split()) for tweet in df['tokenized']]
#remove punctuation

#df['tokenized'] = ["".join([char.lower() for char in df['tokenized'] if char not in string.punctuation])]

#df['tokenized'] = [clean_text(tweet) for tweet in df['tokenized']]

lemmatizer = WordNetLemmatizer()
df['lemmatized'] = [lemmatizer.lemmatize(word) for word in df['tokenized']]



tweet_tokenizer = TweetTokenizer(strip_handles = True, reduce_len=True)
df['tokenized'] = [tweet_tokenizer.tokenize(entry) for entry in df['lemmatized']]
  
df['tokenized'] = [nopunct(tweet) for tweet in df['tokenized']]

#df['tokenized'] = [clean_text(tweet) for tweet in df['tokenized']]

#pun = nltk.RegexpTokenizer(r"\w+")
#df['tokenized'] = df['tokenized'].map(pun.tokenize)
#need to remove www...

#remove punctuation



df.head(20)
    #training a byte-wise bpe tokenizer --> huggingface
    


,Index,message to examine,label (depression result),tokenized,lemmatized
0,106,just had a real good moment. i missssssssss hi...,0,"[just, had, a, real, good, moment, i, misss, h...",just had a real good moment. i missssssssss hi...
1,217,is reading manga http://plurk.com/p/mzp1e,0,"[is, reading, manga]",is reading manga
2,220,@comeagainjen http://twitpic.com/2y2lx - http:...,0,[],@comeagainjen -
3,288,@lapcat Need to send 'em to my accountant tomo...,0,"[Need, to, send, em, to, my, accountant, tomor...",@lapcat Need to send 'em to my accountant tomo...
4,540,ADD ME ON MYSPACE!!! myspace.com/LookThunder,0,"[ADD, ME, ON, MYSPACE]",ADD ME ON MYSPACE!!!
5,624,so sleepy. good times tonight though,0,"[so, sleepy, good, times, tonight, though]",so sleepy. good times tonight though
6,701,"@SilkCharm re: #nbn as someone already said, d...",0,"[re, nbn, as, someone, already, said, does, fi...","@SilkCharm re: nbn as someone already said, do..."
7,808,23 or 24ï¿½C possible today. Nice,0,"[or, ï, ½, C, possible, today, Nice]",or ï¿½C possible today. Nice
8,1193,nite twitterville workout in the am -ciao,0,"[nite, twitterville, workout, in, the, am, ciao]",nite twitterville workout in the am -ciao
9,1324,"@daNanner Night, darlin'! Sweet dreams to you",0,"[Night, darlin, Sweet, dreams, to, you]","@daNanner Night, darlin'! Sweet dreams to you"


In [42]:
data = df['tokenized']
target = df['label (depression result)']
model = Word2Vec(data, window=5, min_count=1, workers=4)

In [43]:
dep = df[df['label (depression result)'] == 1]
depressed_freqdist = FreqDist(dep['tokenized'])
depressed_freqdist.most_common(50)

TypeError: unhashable type: 'list'

In [44]:
model.train(data, total_examples=model.corpus_count, epochs=10)

(1157939, 1424610)

In [45]:
wv = model.wv

In [101]:
wv.most_similar('people')

[('someone', 0.9486861228942871),
 ('things', 0.9418240785598755),
 ('who', 0.9322532415390015),
 ('him', 0.9278783202171326),
 ('they', 0.9197596907615662),
 ('something', 0.9148552417755127),
 ('demand', 0.9143405556678772),
 ('us', 0.908324122428894),
 ('none', 0.9056959748268127),
 ('them', 0.9048210382461548)]

In [46]:
total_vocab = set(word for tweet in data for word in tweet)

In [14]:
len(total_vocab)

18375

In [47]:
glove = {}
with open('glove.twitter.27B.50d.txt', 'rb') as f:
    for line in f:
        parts = line.split()
        word = parts[0].decode('utf-8')
        if word in total_vocab:
            vector = np.array(parts[1:], dtype=np.float32)
            glove[word] = vector

In [17]:
glove['band']

array([-0.3871  ,  0.073452,  0.71934 , -0.13666 ,  0.44342 ,  0.12125 ,
        1.5426  ,  0.69879 ,  0.64324 , -0.092814, -0.024592,  0.31461 ,
       -3.3482  , -0.15844 ,  0.26598 , -0.24607 , -0.31773 ,  0.10577 ,
       -0.14858 ,  0.13287 , -0.26136 ,  1.1169  , -0.6378  , -0.36226 ,
        0.28108 ,  0.34938 , -0.09388 , -0.18142 , -0.334   , -0.17178 ,
        0.28481 , -0.44815 ,  0.46787 , -0.016544,  1.0707  , -0.80139 ,
        0.59951 , -0.66364 ,  0.083816, -0.011978, -1.7931  ,  0.44126 ,
       -1.2955  ,  0.18929 , -1.2393  ,  0.35362 , -0.089281, -0.16474 ,
       -0.18972 , -0.79012 ], dtype=float32)

In [48]:
class W2vVectorizer(object):
    def __init__(self, w2v):
        #takes in a dictionary of words and vectors as input
        self.w2v = w2v
        if len(w2v) == 0:
            self.dimensions = 0
        else:
            self.dimensions = len(w2v[next(iter(glove))])
    
    # Note: Even though it doesn't do anything, it's required that this object implement a fit method or else
    # it can't be used in a scikit-learn pipeline  
    def fit(self, X, y):
        return self
            
    def transform(self, X):
        return np.array([
            np.mean([self.w2v[w] for w in words if w in self.w2v]
                   or [np.zeros(self.dimensions)], axis=0) for words in X])
    


In [49]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

rf =  Pipeline([('Word2Vec Vectorizer', W2vVectorizer(glove)),
              ('Random Forest', RandomForestClassifier(n_estimators=100, verbose=True))])
svc = Pipeline([('Word2Vec Vectorizer', W2vVectorizer(glove)),
                ('Support Vector Machine', SVC())])
lr = Pipeline([('Word2Vec Vectorizer', W2vVectorizer(glove)),
              ('Logistic Regression', LogisticRegression())])

In [50]:
models = [('Random Forest', rf),
          ('Support Vector Machine', svc),
          ('Logistic Regression', lr)]

In [51]:
scores = [(name, cross_val_score(model, data, target, cv=2).mean()) for name, model, in models]


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    2.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    2.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


In [24]:
scores

[('Random Forest', 0.9265076594919527),
 ('Support Vector Machine', 0.9393057979445414),
 ('Logistic Regression', 0.9299980608881133)]

In [53]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dense, LSTM, Embedding
from keras.layers import Dropout, Activation, Bidirectional, GlobalMaxPool1D
from keras.models import Sequential
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.preprocessing import text, sequence

ImportError: Keras requires TensorFlow 2.2 or higher. Install TensorFlow via `pip install tensorflow`